# Introduction to Deep Learning

## Objectives
In this lab, you will embark on the journey of creating a ANN, DNN model tailored for predicting the total expenditure of potential consumers based on various characteristics. As a vehicle salesperson, your goal is to develop a model that can effectively estimate the overall spending potential.

Your task is to build and train an ANN/DNN model using tensorflow in a Jupyter notebook.

Feel Free to Explore the dataset, analyze its contents, and derive meaningful insights. Additionally, feel empowered to create insightful visualizations that enhance the understanding of the data. 

# Step 1: Import Libraries

In [262]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score,mean_absolute_error,mean_squared_error
import time


import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

# Step 2: Load and Explore the Data

In [263]:
df = pd.read_csv('car_purchasing.csv',encoding='latin-1')

# Step 3: Data Cleaning and Preprocessing


**Hint: You could use a `StandardScaler()` or `MinMaxScaler()`**

In [264]:
df.head()

,customer name,customer e-mail,country,gender,age,annual Salary,credit card debt,net worth,car purchase amount
0,Martina Avila,cubilia.Curae.Phasellus@quisaccumsanconvallis.edu,Bulgaria,0,41.851720,62812.09301,11609.380910,238961.2505,35321.45877
1,Harlan Barnes,eu.dolor@diam.co.uk,Belize,0,40.870623,66646.89292,9572.957136,530973.9078,45115.52566
2,Naomi Rodriquez,vulputate.mauris.sagittis@ametconsectetueradip...,Algeria,1,43.152897,53798.55112,11160.355060,638467.1773,42925.70921
3,Jade Cunningham,malesuada@dignissim.com,Cook Islands,1,58.271369,79370.03798,14426.164850,548599.0524,67422.36313
4,Cedric Leach,felis.ullamcorper.viverra@egetmollislectus.net,Brazil,1,57.313749,59729.15130,5358.712177,560304.0671,55915.46248


In [265]:
df.drop(['customer name','customer e-mail'],axis=1,inplace=True)

In [267]:
from sklearn.calibration import LabelEncoder
label_encoder = LabelEncoder()
df['country'] = label_encoder.fit_transform(df['country'])

In [238]:
scaler = MinMaxScaler()

scaled = scaler.fit_transform(df[['annual Salary','credit card debt','net worth']])

df[['annual Salary','credit card debt','net worth']] = scaled

In [239]:
df.head()

,country,gender,age,annual Salary,credit card debt,net worth,car purchase amount
0,27,0,41.851720,0.535151,0.578361,0.223430,35321.45877
1,17,0,40.870623,0.583086,0.476028,0.521402,45115.52566
2,1,1,43.152897,0.422482,0.555797,0.631089,42925.70921
3,41,1,58.271369,0.742125,0.719908,0.539387,67422.36313
4,26,1,57.313749,0.496614,0.264257,0.551331,55915.46248


# Step 4: Train Test Split

In [240]:
X = df.drop('car purchase amount',axis=1)
Y = df['car purchase amount']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


# Step 5: Build the Artifical Neural Network Model

In [241]:
model_ANN = Sequential()

model_ANN.add(Dense(units=32, activation='relu', input_shape=(X_train.shape[1],)))

model_ANN.add(Dense(units=1, activation='linear'))

model_ANN.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=['mse']
)

### Clarify Your Artificial Neural Network (ANN) Model, Optimization, and Loss Function Choices and justify

- Input neurons: 6 features in the dataset.

- Hidden neurons: 32 neaurons.

- Output neurons: 1 neuron for predicting a continuous value.

- Optimizer: Adam: Adaptive Moment Estimation combines aspects of RMSprop and Momentum, adjusting learning rates for each parameter individually.

- Loss function: MSE bc we have Regression Tasks.

- Metrics: MSE monitoring for model performance in Regression Tasks.


# Step 6: Train the Model


In [242]:
start_time_ANN = time.time()
model_ANN.fit(X_train,Y_train, epochs=50, batch_size=32, validation_split=0.2)
training_time_ANN = time.time() - start_time_ANN


Epoch 1/50


10/10 [==============================] - 1s 22ms/step - loss: 2074318080.0000 - mse: 2074318080.0000 - val_loss: 1936686080.0000 - val_mse: 1936686080.0000
Epoch 2/50
10/10 [==============================] - 0s 6ms/step - loss: 2073972992.0000 - mse: 2073972992.0000 - val_loss: 1936334464.0000 - val_mse: 1936334464.0000
Epoch 3/50
10/10 [==============================] - 0s 6ms/step - loss: 2073616000.0000 - mse: 2073616000.0000 - val_loss: 1935962368.0000 - val_mse: 1935962368.0000
Epoch 4/50
10/10 [==============================] - 0s 11ms/step - loss: 2073230720.0000 - mse: 2073230720.0000 - val_loss: 1935564800.0000 - val_mse: 1935564800.0000
Epoch 5/50
10/10 [==============================] - 0s 6ms/step - loss: 2072811776.0000 - mse: 2072811776.0000 - val_loss: 1935134336.0000 - val_mse: 1935134336.0000
Epoch 6/50
10/10 [==============================] - 0s 6ms/step - loss: 2072368512.0000 - mse: 2072368512.0000 - val_loss: 1934649728.0000 - val_mse: 1934649728.0000
Epoch 7/50
10

# Step 7: Evaluate the Model

In [243]:
start_time_ANN = time.time()
Y_pred_ANN = model_ANN.predict(X_test)
prediction_time_ANN = time.time() - start_time_ANN


4/4 [==============================] - 0s 2ms/step


In [244]:
mse_ANN = mean_squared_error(Y_test, Y_pred_ANN)
mae_ANN = mean_absolute_error(Y_test,Y_pred_ANN)

print(f'MSE: {mse_ANN:.2f}')
print(f'MAE: {mae_ANN:.2f}')


MSE: 2044589132.72
MAE: 44008.64


# Step 8: Build the Deep Neural Network Model

In [245]:
model_DNN = Sequential()
model_DNN.add(Dense(units=32, activation='relu', input_shape=(X_train.shape[1],)))

model_DNN.add(Dense(units=32, activation='relu'))
model_DNN.add(Dense(units=32, activation='relu'))

model_DNN.add(Dense(units=1, activation='linear'))

model_DNN.compile(
                optimizer='adam',
                   loss='mean_squared_error',
                     metrics=['mse']

)


### Clarify Your Deep Neural Network (DNN) Model, Optimization, and Loss Function Choices and justify 

- Input neurons: 6 features in the dataset.

- Hidden neurons: we have 3 layers are hidden, and each layer has 32 neaurons.

- Output neurons: 1 neauron for predicting a continuous value.

- Optimizer: Adam: Adaptive Moment Estimation combines aspects of RMSprop and Momentum, adjusting learning rates for each parameter individually.

- Loss function: MSE bc we have Regression Tasks.

- Metrics: MSE monitoring for model performance in Regression Tasks.


# Step 9: Train the Model

In [246]:
start_time_DNN = time.time()
model_DNN.fit(X_train,Y_train, epochs=50, batch_size=32, validation_split=0.2)
training_time_DNN = time.time() - start_time_DNN


Epoch 1/50


10/10 [==============================] - 1s 24ms/step - loss: 2076064512.0000 - mse: 2076064512.0000 - val_loss: 1937632512.0000 - val_mse: 1937632512.0000
Epoch 2/50
10/10 [==============================] - 0s 6ms/step - loss: 2074331136.0000 - mse: 2074331136.0000 - val_loss: 1935969024.0000 - val_mse: 1935969024.0000
Epoch 3/50
10/10 [==============================] - 0s 6ms/step - loss: 2072581504.0000 - mse: 2072581504.0000 - val_loss: 1933977344.0000 - val_mse: 1933977344.0000
Epoch 4/50
10/10 [==============================] - 0s 6ms/step - loss: 2070187648.0000 - mse: 2070187648.0000 - val_loss: 1930929536.0000 - val_mse: 1930929536.0000
Epoch 5/50
10/10 [==============================] - 0s 6ms/step - loss: 2066151424.0000 - mse: 2066151424.0000 - val_loss: 1925759744.0000 - val_mse: 1925759744.0000
Epoch 6/50
10/10 [==============================] - 0s 7ms/step - loss: 2059719424.0000 - mse: 2059719424.0000 - val_loss: 1917663488.0000 - val_mse: 1917663488.0000
Epoch 7/50
10/

# Step 10: Evaluate the Model

In [247]:
start_time_DNN = time.time()
Y_pred_DNN = model_DNN.predict(X_test)
prediction_time_DNN = time.time() - start_time_DNN

4/4 [==============================] - 0s 2ms/step


In [248]:
mse_DNN = mean_squared_error(Y_test, Y_pred_DNN)
mae_DNN = mean_absolute_error(Y_test,Y_pred_DNN)

print(f'MSE: {mse_DNN:.2f}')
print(f'MAE: {mae_DNN:.2f}')


MSE: 183962920.87
MAE: 11600.14


# Step 11: Evaluate and Compare Scores, Training Time, and Prediction Time of ANN/DNN Models

In [253]:

print("ANN Metrics:")
print(f"Mean Squared Error: {mse_ANN:.2f}")
print(f"Mean Absolute Error: {mae_ANN:.2f}")

print("\nDNN Metrics:")
print(f"Mean Squared Error: {mse_DNN:.2f}")
print(f"Mean Absolute Error: {mae_DNN:.2f}")

print("\nTraining Time:")
print(f"ANN Training Time:{training_time_ANN:.4f}", "seconds")
print(f"DNN Training Time: {training_time_DNN:.4f}", "seconds")

print("\nPrediction Time:")
print(f"ANN Prediction Time: {prediction_time_ANN:.4f}", "seconds")
print(f"DNN Prediction Time: {prediction_time_DNN:.4f}", "seconds")

ANN Metrics:
Mean Squared Error: 2044589132.72
Mean Absolute Error: 44008.64

DNN Metrics:
Mean Squared Error: 183962920.87
Mean Absolute Error: 11600.14

Training Time:
ANN Training Time:4.1567 seconds
DNN Training Time: 4.4311 seconds

Prediction Time:
ANN Prediction Time: 0.1449 seconds
DNN Prediction Time: 0.1493 seconds
